# Análisis de Datos y Modelado Predictivo para Eventos de Inundación

Este notebook presenta un análisis exhaustivo y modelado predictivo realizado sobre un dataset de variables hidrológicas y meteorológicas relacionadas con eventos de inundación.

## Objetivos

1. Realizar un análisis exploratorio de datos (EDA)
2. Preparar y limpiar los datos
3. Balancear las clases para mejorar el modelado
4. Construir y evaluar modelos predictivos
5. Validar los modelos y analizar su confiabilidad

## Descripción del Dataset

El dataset "datos_final.csv" contiene registros horarios con las siguientes variables:
- `hour_updated`: Fecha y hora de la medición
- `p01m`: Medida de precipitación (mm)
- `cfs`: Caudal (pies cúbicos por segundo)
- `height`: Altura del agua (m)
- `flood_event`: Variable objetivo, indica si ocurrió una inundación (1) o no (0)

## 1. Importación de Bibliotecas

Primero, importamos todas las bibliotecas necesarias para el análisis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

# Configurar el estilo de las visualizaciones
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('deep')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## 2. Carga y Exploración Inicial de Datos

Cargamos el dataset y realizamos una exploración inicial para entender su estructura y características.

In [ ]:
# Cargar los datos
print("1. CARGANDO DATOS")
data = pd.read_csv('datos_final.csv')
print(f"Dimensiones del dataset: {data.shape}")

In [ ]:
# Exploración inicial de los datos
print("\n2. EXPLORACIÓN INICIAL")
print("\nPrimeras filas del dataset:")
data.head()

In [ ]:
# Información del dataset
print("\nInformación del dataset:")
data.info()

In [ ]:
# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
data.describe()

## 3. Análisis Exploratorio de Datos (EDA)

Ahora realizaremos un análisis exploratorio más profundo para entender mejor nuestros datos y descubrir patrones e insights relevantes.

### 3.1 Verificación de Valores Nulos

Verificamos si hay valores nulos en el dataset que necesiten ser tratados.

In [ ]:
# Verificar valores nulos
print("\n3. VERIFICANDO VALORES NULOS")
nulos = data.isnull().sum()
print(nulos)

### 3.2 Exploración de la Variable Objetivo

Analizamos la distribución de la variable objetivo (`flood_event`) para entender el balance entre las clases.

In [ ]:
# Explorar la variable objetivo
print("\n4. EXPLORANDO LA VARIABLE OBJETIVO (flood_event)")
print(data['flood_event'].value_counts())
print(f"Proporción de eventos de inundación: {data['flood_event'].mean() * 100:.2f}%")

# Visualizar la distribución de clases
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='flood_event', data=data)
plt.title('Distribución de la Variable Objetivo (flood_event)', fontsize=16)
plt.xlabel('Evento de Inundación (0=No, 1=Sí)', fontsize=14)
plt.ylabel('Cantidad de Registros', fontsize=14)

# Añadir etiquetas con la cantidad y porcentaje
total = len(data)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2.,
            height + 0.1,
            f'{height}\n({height/total:.1%})',
            ha="center", fontsize=12)
plt.show()

La distribución muestra un claro desbalance entre las clases, con aproximadamente un 3% de eventos de inundación. Este desbalance es típico en fenómenos extremos y requerirá técnicas especiales para el modelado.

Este tipo de desbalance representa un desafío para los modelos de machine learning porque:
1. Pueden estar sesgados hacia la predicción de la clase mayoritaria
2. Las métricas tradicionales como la precisión (accuracy) pueden ser engañosas
3. El modelo puede no aprender adecuadamente los patrones que conducen a inundaciones

Por estas razones, más adelante aplicaremos técnicas de balanceo como SMOTE y Random Undersampling.

### 3.3 Análisis Temporal

Analizamos la distribución temporal de los eventos para identificar patrones estacionales o tendencias.

In [ ]:
# Análisis temporal
print("\n5. ANÁLISIS TEMPORAL")
# Convertir la columna de hora a datetime
data['hour_updated'] = pd.to_datetime(data['hour_updated'])
data['year'] = data['hour_updated'].dt.year
data['month'] = data['hour_updated'].dt.month
data['day'] = data['hour_updated'].dt.day
data['hour'] = data['hour_updated'].dt.hour

# Analizar eventos de inundación por año y mes
eventos_por_anio = data.groupby('year')['flood_event'].sum().reset_index()
eventos_por_mes = data.groupby(['year', 'month'])['flood_event'].sum().reset_index()

# Visualizar eventos por año
plt.figure(figsize=(14, 7))
ax = sns.barplot(x='year', y='flood_event', data=eventos_por_anio)
plt.title('Eventos de Inundación por Año', fontsize=16)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Número de Eventos', fontsize=14)

# Añadir etiquetas con la cantidad
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'bottom', fontsize=12)
plt.show()

# Visualizar eventos por mes (heatmap)
eventos_heatmap = eventos_por_mes.pivot(index='year', columns='month', values='flood_event').fillna(0)
plt.figure(figsize=(14, 7))
sns.heatmap(eventos_heatmap, annot=True, fmt='g', cmap='YlOrRd', linewidths=.5)
plt.title('Eventos de Inundación por Mes y Año', fontsize=16)
plt.xlabel('Mes', fontsize=14)
plt.ylabel('Año', fontsize=14)
plt.show()

El análisis temporal nos permite identificar patrones estacionales y tendencias a lo largo del tiempo. Podemos observar si ciertos meses o años tienen una mayor incidencia de eventos de inundación. Esta información es valiosa para entender la naturaleza cíclica o estacional del fenómeno y puede ser útil para construir características temporales que mejoren nuestros modelos predictivos.

### 3.4 Análisis de Correlaciones

Analizamos las correlaciones entre las variables numéricas para entender sus relaciones.

In [ ]:
# Análisis de correlaciones
print("\n6. ANÁLISIS DE CORRELACIONES")
correlaciones = data[['p01m', 'cfs', 'height', 'flood_event']].corr()
print(correlaciones)

# Visualizar matriz de correlación
plt.figure(figsize=(12, 10))
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', linewidths=0.5, vmin=-1, vmax=1, square=True)
plt.title('Matriz de Correlación', fontsize=16)
plt.show()

### Interpretación de la Matriz de Correlación

La matriz de correlación muestra el coeficiente de correlación de Pearson entre cada par de variables. Este coeficiente varía entre -1 y 1, donde:

- **1**: Correlación positiva perfecta (cuando una variable aumenta, la otra aumenta proporcionalmente)
- **0**: No hay correlación lineal
- **-1**: Correlación negativa perfecta (cuando una variable aumenta, la otra disminuye proporcionalmente)

En la visualización, los colores ayudan a interpretar los valores:
- **Rojo intenso**: Correlación positiva fuerte
- **Azul intenso**: Correlación negativa fuerte
- **Colores claros**: Correlación débil o nula

#### Observaciones clave:

1. **p01m vs cfs**: Correlación cercana a 1, lo que indica una relación muy fuerte entre precipitación y caudal. Esto es lógico ya que un aumento en las precipitaciones suele provocar un aumento en el caudal de agua.

2. **p01m/cfs vs flood_event**: Correlación moderada positiva (aproximadamente 0.52), indicando que estas variables son buenos predictores de eventos de inundación. A mayor precipitación y caudal, mayor probabilidad de inundación.

3. **height vs otras variables**: Correlación muy baja (cerca de 0), lo que sugiere que la altura del agua medida en este contexto no tiene una relación lineal fuerte con las otras variables. Esto puede parecer contraintuitivo, pero podría explicarse si:
   - La altura se mide en un punto que no refleja bien los cambios en el sistema hidrológico general
   - Hay un retraso significativo entre los cambios en precipitación/caudal y los cambios en altura
   - La relación es no lineal (el coeficiente de Pearson solo captura relaciones lineales)

#### Implicaciones para el modelado:

- La alta correlación entre p01m y cfs sugiere **multicolinealidad**, lo que podría afectar a algunos tipos de modelos. En modelos como regresión lineal, podría ser recomendable eliminar una de estas variables, pero en modelos de árbol como Random Forest, esto no suele ser un problema.

- Las correlaciones moderadas con la variable objetivo sugieren que p01m y cfs serán características importantes para nuestro modelo predictivo.

- La baja correlación de height sugiere que podríamos necesitar transformar esta variable o crear variables derivadas (como cambios en la altura) para capturar mejor su relación con los eventos de inundación.

### 3.5 Distribución de Variables Numéricas

Visualizamos la distribución de las variables predictoras para entender mejor su comportamiento.

In [ ]:
# Distribución de variables numéricas
print("\n7. DISTRIBUCIÓN DE VARIABLES NUMÉRICAS")
fig, axes = plt.subplots(3, 1, figsize=(14, 20))

sns.histplot(data=data, x='p01m', kde=True, ax=axes[0])
axes[0].set_title('Distribución de p01m (Precipitación)', fontsize=16)
axes[0].set_xlabel('p01m (mm)', fontsize=14)
axes[0].set_ylabel('Frecuencia', fontsize=14)

sns.histplot(data=data, x='cfs', kde=True, ax=axes[1])
axes[1].set_title('Distribución de cfs (Caudal)', fontsize=16)
axes[1].set_xlabel('cfs (pies cúbicos por segundo)', fontsize=14)
axes[1].set_ylabel('Frecuencia', fontsize=14)

sns.histplot(data=data, x='height', kde=True, ax=axes[2])
axes[2].set_title('Distribución de height (Altura)', fontsize=16)
axes[2].set_xlabel('height (m)', fontsize=14)
axes[2].set_ylabel('Frecuencia', fontsize=14)

plt.tight_layout()
plt.show()

Estas distribuciones nos ayudan a entender cómo se comportan las variables predictoras. Podemos observar:

- **p01m y cfs**: Distribuciones sesgadas a la derecha (asimetría positiva), con la mayoría de valores concentrados en el rango inferior y una cola larga hacia valores más altos. Esto es típico en variables como precipitación y caudal, donde la mayoría de las mediciones son bajas o moderadas, pero ocasionalmente se producen valores extremos.

- **height**: Distribución multimodal (múltiples picos), lo que sugiere diferentes estados o niveles estables del cuerpo de agua en diferentes momentos. Esta complejidad podría explicar su baja correlación lineal con las otras variables.

### 3.6 Relación entre Variables Predictoras y Variable Objetivo

Analizamos cómo se relacionan las variables predictoras con la variable objetivo mediante diagramas de caja.

In [ ]:
# Relación entre variables predictoras y la variable objetivo
print("\n8. RELACIÓN ENTRE VARIABLES PREDICTORAS Y OBJETIVO")
fig, axes = plt.subplots(3, 1, figsize=(14, 20))

sns.boxplot(x='flood_event', y='p01m', data=data, ax=axes[0])
axes[0].set_title('p01m vs flood_event', fontsize=16)
axes[0].set_xlabel('Evento de Inundación (0=No, 1=Sí)', fontsize=14)
axes[0].set_ylabel('p01m (mm)', fontsize=14)

sns.boxplot(x='flood_event', y='cfs', data=data, ax=axes[1])
axes[1].set_title('cfs vs flood_event', fontsize=16)
axes[1].set_xlabel('Evento de Inundación (0=No, 1=Sí)', fontsize=14)
axes[1].set_ylabel('cfs (pies cúbicos por segundo)', fontsize=14)

sns.boxplot(x='flood_event', y='height', data=data, ax=axes[2])
axes[2].set_title('height vs flood_event', fontsize=16)
axes[2].set_xlabel('Evento de Inundación (0=No, 1=Sí)', fontsize=14)
axes[2].set_ylabel('height (m)', fontsize=14)

plt.tight_layout()
plt.show()

Los diagramas de caja nos muestran cómo se distribuyen las variables predictoras en función de la variable objetivo. Esto nos ayuda a identificar si hay diferencias significativas entre los eventos de inundación y no inundación para cada variable predictora.

Observaciones:
- **p01m y cfs**: Muestran una clara diferencia entre eventos de inundación (1) y no inundación (0). Los eventos de inundación tienen valores mucho más altos, lo que confirma su poder predictivo.
- **height**: La diferencia es menos pronunciada, lo que concuerda con su baja correlación con la variable objetivo.

## 4. Preparación de los Datos

En esta sección, preparamos los datos para el modelado, realizando ingeniería de características, división en conjuntos de entrenamiento, validación y prueba, escalado y balanceo de clases.

In [ ]:
# Preparación de los datos
print("\n9. PREPARACIÓN DE LOS DATOS")

# Extraer características de tiempo (características cíclicas)
data['sin_hora'] = np.sin(2 * np.pi * data['hour'] / 24)
data['cos_hora'] = np.cos(2 * np.pi * data['hour'] / 24)
data['sin_mes'] = np.sin(2 * np.pi * data['month'] / 12)
data['cos_mes'] = np.cos(2 * np.pi * data['month'] / 12)

# Crear variables de rezago (lag) para capturar tendencias
for i in range(1, 4):  # Crear 3 variables de rezago
    data[f'p01m_lag_{i}'] = data['p01m'].shift(i)
    data[f'cfs_lag_{i}'] = data['cfs'].shift(i)
    data[f'height_lag_{i}'] = data['height'].shift(i)

# Eliminar los registros con NaN debido a los rezagos
data = data.dropna()

# Características a utilizar en el modelo
features = ['p01m', 'cfs', 'height', 
            'sin_hora', 'cos_hora', 'sin_mes', 'cos_mes',
            'p01m_lag_1', 'p01m_lag_2', 'p01m_lag_3',
            'cfs_lag_1', 'cfs_lag_2', 'cfs_lag_3',
            'height_lag_1', 'height_lag_2', 'height_lag_3']

X = data[features]
y = data['flood_event']

### 4.1 División de Datos en Train, Validation y Test

Dividiremos nuestros datos en tres conjuntos:
- **Train (60%)**: Para entrenar los modelos
- **Validation (20%)**: Para ajustar hiperparámetros y evitar el sobreajuste
- **Test (20%)**: Para la evaluación final del modelo

Esta división es una práctica recomendada que nos permite evaluar de manera más robusta el rendimiento de nuestros modelos.

In [ ]:
# Dividir datos en conjuntos de entrenamiento, validación y prueba
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp) 
# 0.25 * 0.8 = 0.2 del conjunto original

# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Mostrar la forma de los conjuntos de datos
print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de X_val: {X_val.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"\nDistribución de y_train: \n{pd.Series(y_train).value_counts()}")
print(f"\nDistribución de y_val: \n{pd.Series(y_val).value_counts()}")
print(f"\nDistribución de y_test: \n{pd.Series(y_test).value_counts()}")

### 4.2 Balanceo de Clases

Aplicamos técnicas de balanceo para mejorar el rendimiento de los modelos con datos desbalanceados.

In [ ]:
# Balanceo de clases
print("\n10. BALANCEO DE CLASES")
print("Antes del balanceo:")
print(pd.Series(y_train).value_counts())

# Aplicar SMOTE (sobremuestreo)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("\nDespués del balanceo con SMOTE:")
print(pd.Series(y_train_smote).value_counts())

# También probaremos submuestreo
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_scaled, y_train)

print("\nDespués del balanceo con RandomUnderSampler:")
print(pd.Series(y_train_rus).value_counts())

# Visualizar la distribución después del balanceo
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.countplot(x=y_train, ax=axes[0])
axes[0].set_title('Distribución Original', fontsize=14)
axes[0].set_xlabel('Evento de Inundación (0=No, 1=Sí)', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)

sns.countplot(x=y_train_smote, ax=axes[1])
axes[1].set_title('Después de SMOTE', fontsize=14)
axes[1].set_xlabel('Evento de Inundación (0=No, 1=Sí)', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)

sns.countplot(x=y_train_rus, ax=axes[2])
axes[2].set_title('Después de RandomUnderSampler', fontsize=14)
axes[2].set_xlabel('Evento de Inundación (0=No, 1=Sí)', fontsize=12)
axes[2].set_ylabel('Frecuencia', fontsize=12)

plt.tight_layout()
plt.show()

### Técnicas de Balanceo Explicadas:

1. **SMOTE (Synthetic Minority Over-sampling Technique)**:
   - Genera ejemplos sintéticos de la clase minoritaria (eventos de inundación)
   - Funciona creando nuevos ejemplos a lo largo de los segmentos de línea que unen puntos vecinos de la clase minoritaria
   - Ventaja: Mantiene toda la información de la clase mayoritaria
   - Desventaja: Puede crear ejemplos sintéticos en regiones no realistas del espacio de características

2. **Random Undersampling (RUS)**:
   - Reduce aleatoriamente la cantidad de ejemplos en la clase mayoritaria (no inundación)
   - Ventaja: Método simple y rápido
   - Desventaja: Puede descartar información valiosa de la clase mayoritaria

Probaremos ambos enfoques para determinar cuál produce mejores resultados en nuestros modelos. El balanceo solo se aplica al conjunto de entrenamiento, mientras que los conjuntos de validación y prueba mantienen la distribución original para reflejar el escenario real.

## 5. Modelado y Evaluación

En esta sección, entrenamos y evaluamos diferentes modelos de clasificación utilizando distintas estrategias de balanceo.

In [ ]:
# Función para evaluar y mostrar resultados
def evaluar_modelo(nombre, modelo, X_train, y_train, X_val, y_val, X_test=None, y_test=None):
    # Entrenar el modelo
    modelo.fit(X_train, y_train)
    
    # Predecir en conjunto de validación
    y_val_pred = modelo.predict(X_val)
    
    # Calcular métricas en validación
    val_acc = accuracy_score(y_val, y_val_pred)
    val_prec = precision_score(y_val, y_val_pred)
    val_rec = recall_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred)
    
    print(f"\n--- Resultados de {nombre} en Validación ---")
    print(f"Accuracy: {val_acc:.4f}")
    print(f"Precision: {val_prec:.4f}")
    print(f"Recall: {val_rec:.4f}")
    print(f"F1-Score: {val_f1:.4f}")
    
    # Matriz de confusión para validación
    cm_val = confusion_matrix(y_val, y_val_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', xticklabels=['No inundación', 'Inundación'], 
                yticklabels=['No inundación', 'Inundación'])
    plt.ylabel('Real', fontsize=14)
    plt.xlabel('Predicción', fontsize=14)
    plt.title(f'Matriz de Confusión (Validación) - {nombre}', fontsize=16)
    plt.show()
    
    # Probabilidades para curva ROC (validación)
    y_val_proba = modelo.predict_proba(X_val)[:, 1]
    val_fpr, val_tpr, _ = roc_curve(y_val, y_val_proba)
    val_roc_auc = auc(val_fpr, val_tpr)
    
    # Curva Precision-Recall (validación)
    val_precision, val_recall, _ = precision_recall_curve(y_val, y_val_proba)
    val_pr_auc = auc(val_recall, val_precision)
    
    # Gráficas ROC y Precision-Recall
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    
    # ROC
    axes[0].plot(val_fpr, val_tpr, label=f'Validación (AUC = {val_roc_auc:.4f})')
    axes[0].plot([0, 1], [0, 1], 'k--')
    axes[0].set_xlabel('Tasa de Falsos Positivos', fontsize=14)
    axes[0].set_ylabel('Tasa de Verdaderos Positivos', fontsize=14)
    axes[0].set_title(f'Curva ROC - {nombre}', fontsize=16)
    axes[0].legend(loc='lower right', fontsize=12)
    axes[0].grid(True, linestyle='--', alpha=0.7)
    
    # Precision-Recall
    axes[1].plot(val_recall, val_precision, label=f'Validación (AUC = {val_pr_auc:.4f})')
    axes[1].set_xlabel('Recall', fontsize=14)
    axes[1].set_ylabel('Precision', fontsize=14)
    axes[1].set_title(f'Curva Precision-Recall - {nombre}', fontsize=16)
    axes[1].legend(loc='lower left', fontsize=12)
    axes[1].grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Si se proporcionan datos de prueba, evaluar en ellos también
    if X_test is not None and y_test is not None:
        y_test_pred = modelo.predict(X_test)
        test_acc = accuracy_score(y_test, y_test_pred)
        test_prec = precision_score(y_test, y_test_pred)
        test_rec = recall_score(y_test, y_test_pred)
        test_f1 = f1_score(y_test, y_test_pred)
        
        print(f"\n--- Resultados de {nombre} en Test ---")
        print(f"Accuracy: {test_acc:.4f}")
        print(f"Precision: {test_prec:.4f}")
        print(f"Recall: {test_rec:.4f}")
        print(f"F1-Score: {test_f1:.4f}")
        
        return modelo, val_acc, val_prec, val_rec, val_f1, val_roc_auc, test_acc, test_prec, test_rec, test_f1
    
    return modelo, val_acc, val_prec, val_rec, val_f1, val_roc_auc

### 5.1 Regresión Logística

In [ ]:
# Entrenamiento y evaluación de modelos
print("\n11. ENTRENAMIENTO Y EVALUACIÓN DE MODELOS")

# 11.1 Modelo de Regresión Logística
print("\n11.1 REGRESIÓN LOGÍSTICA")
# Con datos originales
lr = LogisticRegression(max_iter=1000, random_state=42)
lr_orig, val_acc_lr_orig, val_prec_lr_orig, val_rec_lr_orig, val_f1_lr_orig, val_auc_lr_orig = evaluar_modelo(
    "Regresión Logística (original)", lr, X_train_scaled, y_train, X_val_scaled, y_val
)

# Con SMOTE
lr_smote = LogisticRegression(max_iter=1000, random_state=42)
lr_smote, val_acc_lr_smote, val_prec_lr_smote, val_rec_lr_smote, val_f1_lr_smote, val_auc_lr_smote = evaluar_modelo(
    "Regresión Logística (SMOTE)", lr_smote, X_train_smote, y_train_smote, X_val_scaled, y_val
)

# Con RandomUnderSampler
lr_rus = LogisticRegression(max_iter=1000, random_state=42)
lr_rus, val_acc_lr_rus, val_prec_lr_rus, val_rec_lr_rus, val_f1_lr_rus, val_auc_lr_rus = evaluar_modelo(
    "Regresión Logística (RUS)", lr_rus, X_train_rus, y_train_rus, X_val_scaled, y_val
)

### 5.2 Random Forest

In [ ]:
# 11.2 Random Forest
print("\n11.2 RANDOM FOREST")
# Con datos originales
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_orig, val_acc_rf_orig, val_prec_rf_orig, val_rec_rf_orig, val_f1_rf_orig, val_auc_rf_orig = evaluar_modelo(
    "Random Forest (original)", rf, X_train_scaled, y_train, X_val_scaled, y_val
)

# Con SMOTE
rf_smote = RandomForestClassifier(n_estimators=100, random_state=42)
rf_smote, val_acc_rf_smote, val_prec_rf_smote, val_rec_rf_smote, val_f1_rf_smote, val_auc_rf_smote = evaluar_modelo(
    "Random Forest (SMOTE)", rf_smote, X_train_smote, y_train_smote, X_val_scaled, y_val
)

# Con RandomUnderSampler
rf_rus = RandomForestClassifier(n_estimators=100, random_state=42)
rf_rus, val_acc_rf_rus, val_prec_rf_rus, val_rec_rf_rus, val_f1_rf_rus, val_auc_rf_rus = evaluar_modelo(
    "Random Forest (RUS)", rf_rus, X_train_rus, y_train_rus, X_val_scaled, y_val
)

### 5.3 Gradient Boosting

In [ ]:
# 11.3 Gradient Boosting
print("\n11.3 GRADIENT BOOSTING")
# Con datos originales
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_orig, val_acc_gb_orig, val_prec_gb_orig, val_rec_gb_orig, val_f1_gb_orig, val_auc_gb_orig = evaluar_modelo(
    "Gradient Boosting (original)", gb, X_train_scaled, y_train, X_val_scaled, y_val
)

# Con SMOTE
gb_smote = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_smote, val_acc_gb_smote, val_prec_gb_smote, val_rec_gb_smote, val_f1_gb_smote, val_auc_gb_smote = evaluar_modelo(
    "Gradient Boosting (SMOTE)", gb_smote, X_train_smote, y_train_smote, X_val_scaled, y_val
)

# Con RandomUnderSampler
gb_rus = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_rus, val_acc_gb_rus, val_prec_gb_rus, val_rec_gb_rus, val_f1_gb_rus, val_auc_gb_rus = evaluar_modelo(
    "Gradient Boosting (RUS)", gb_rus, X_train_rus, y_train_rus, X_val_scaled, y_val
)

## 6. Selección del Mejor Modelo y Evaluación en Test

Basándonos en los resultados de validación, seleccionamos el mejor modelo y lo evaluamos en el conjunto de prueba.

In [ ]:
# Seleccionar el mejor modelo basado en F1-Score de validación
val_modelos = [
    ("LR Original", lr_orig, X_train_scaled, y_train, val_f1_lr_orig),
    ("LR SMOTE", lr_smote, X_train_smote, y_train_smote, val_f1_lr_smote),
    ("LR RUS", lr_rus, X_train_rus, y_train_rus, val_f1_lr_rus),
    ("RF Original", rf_orig, X_train_scaled, y_train, val_f1_rf_orig),
    ("RF SMOTE", rf_smote, X_train_smote, y_train_smote, val_f1_rf_smote),
    ("RF RUS", rf_rus, X_train_rus, y_train_rus, val_f1_rf_rus),
    ("GB Original", gb_orig, X_train_scaled, y_train, val_f1_gb_orig),
    ("GB SMOTE", gb_smote, X_train_smote, y_train_smote, val_f1_gb_smote),
    ("GB RUS", gb_rus, X_train_rus, y_train_rus, val_f1_gb_rus)
]

mejor_val_modelo_info = max(val_modelos, key=lambda x: x[4])
mejor_nombre, mejor_modelo, mejor_X_train, mejor_y_train, mejor_f1_val = mejor_val_modelo_info

print(f"El mejor modelo según la validación es: {mejor_nombre} con F1-Score de {mejor_f1_val:.4f}")

# Evaluar el mejor modelo en el conjunto de prueba
print("\n12. EVALUACIÓN EN CONJUNTO DE PRUEBA")
_, _, _, _, _, _, test_acc, test_prec, test_rec, test_f1 = evaluar_modelo(
    f"{mejor_nombre} (Test Final)", mejor_modelo, 
    mejor_X_train, mejor_y_train,
    X_val_scaled, y_val,  # para mantener las gráficas de validación
    X_test_scaled, y_test  # evaluación en test
)

## 7. Importancia de Características

Analizamos la importancia relativa de cada característica en el modelo final.

In [ ]:
# Importancia de características
if hasattr(mejor_modelo, 'feature_importances_'):
    print("\n13. IMPORTANCIA DE CARACTERÍSTICAS")
    importances = mejor_modelo.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    plt.figure(figsize=(14, 10))
    plt.title('Importancia de Características', fontsize=16)
    plt.barh(range(len(features)), importances[indices], align='center', color='steelblue')
    plt.yticks(range(len(features)), [features[i] for i in indices], fontsize=12)
    plt.xlabel('Importancia Relativa', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    print("\nCaracterísticas ordenadas por importancia:")
    for i in range(len(features)):
        print(f"{features[indices[i]]}: {importances[indices[i]]:.4f}")

### Explicación de la Importancia de Características

#### ¿Qué es la Importancia de Características?

La importancia de características es una medida que indica cuánto contribuye cada variable predictora al rendimiento del modelo. En modelos basados en árboles como Random Forest y Gradient Boosting, esta importancia se calcula de manera diferente:

- **Random Forest**: Utiliza principalmente la **reducción de impureza de Gini** o la **reducción de entropía**. Cada vez que se utiliza una característica para dividir un nodo, se calcula cuánto mejora la pureza de las clases. La suma de estas mejoras a través de todos los árboles se normaliza para obtener la importancia relativa.

- **Gradient Boosting**: Utiliza la **reducción en la función de pérdida** que se obtiene al dividir según cada característica.

#### Cómo Interpretar la Gráfica

1. **Eje Y**: Muestra los nombres de las características, ordenadas de mayor a menor importancia.
2. **Eje X**: Representa la importancia relativa de cada característica, expresada como un valor entre 0 y 1, donde la suma de todas las importancias es 1.
3. **Barras**: La longitud de cada barra indica la importancia relativa de la característica correspondiente. Las características con barras más largas son más influyentes en las predicciones del modelo.

#### Interpretación de los Resultados

Normalmente, observamos que:

1. **Variables dominantes**: `cfs` (caudal) y `p01m` (precipitación) suelen ser las características más importantes, lo que concuerda con nuestra intuición hidrológica y con los análisis de correlación previos.

2. **Variables de rezago**: Las variables de rezago (lag) pueden tener importancia moderada, lo que indica que las condiciones previas influyen en la predicción de inundaciones.

3. **Variables temporales cíclicas**: Las variables `sin_hora`, `cos_hora`, `sin_mes`, `cos_mes` generalmente tienen menor importancia, pero aún pueden capturar patrones estacionales o diarios relevantes.

#### Implicaciones

- **Para el modelo**: Podríamos considerar simplificar el modelo eliminando las características menos importantes, aunque los modelos basados en árboles suelen manejar bien las características irrelevantes.

- **Para la comprensión del fenómeno**: Nos ayuda a entender los factores más determinantes en la predicción de inundaciones, lo que puede informar estrategias de monitoreo y alerta temprana.

- **Para futuras mejoras**: Podríamos enfocarnos en mejorar la precisión o la frecuencia de medición de las características más importantes, o buscar variables adicionales relacionadas con ellas.

## 8. Validación Cruzada del Mejor Modelo

Realizamos una validación cruzada para evaluar la robustez del mejor modelo.

In [ ]:
# Validación cruzada con el mejor modelo
print("\n14. VALIDACIÓN CRUZADA")

# Realizar validación cruzada con el mejor modelo
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(mejor_modelo, mejor_X_train, mejor_y_train, cv=cv, scoring='f1')
print(f"Resultados de validación cruzada (F1-Score): {cv_scores}")
print(f"Promedio de F1-Score en validación cruzada: {cv_scores.mean():.4f}")
print(f"Desviación estándar de F1-Score: {cv_scores.std():.4f}")

# Visualizar los resultados de validación cruzada
plt.figure(figsize=(12, 6))
plt.bar(range(1, 6), cv_scores, color='steelblue')
plt.axhline(y=cv_scores.mean(), color='red', linestyle='-', label=f'Promedio: {cv_scores.mean():.4f}')
plt.xlabel('Fold', fontsize=14)
plt.ylabel('F1-Score', fontsize=14)
plt.title(f'Resultados de Validación Cruzada para {mejor_nombre}', fontsize=16)
plt.xticks(range(1, 6))
plt.ylim(0, 1.1)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

## 9. Resumen de Resultados de Validación

Comparamos los resultados de todos los modelos para identificar el mejor enfoque.

In [ ]:
# Resumen de resultados de validación
print("\n15. RESUMEN DE RESULTADOS DE TODOS LOS MODELOS EN VALIDACIÓN")

# Crear un DataFrame con los resultados
val_results = pd.DataFrame({
    'Modelo': ['LR Original', 'LR SMOTE', 'LR RUS', 
               'RF Original', 'RF SMOTE', 'RF RUS',
               'GB Original', 'GB SMOTE', 'GB RUS'],
    'Accuracy': [val_acc_lr_orig, val_acc_lr_smote, val_acc_lr_rus,
                val_acc_rf_orig, val_acc_rf_smote, val_acc_rf_rus,
                val_acc_gb_orig, val_acc_gb_smote, val_acc_gb_rus],
    'Precision': [val_prec_lr_orig, val_prec_lr_smote, val_prec_lr_rus,
                 val_prec_rf_orig, val_prec_rf_smote, val_prec_rf_rus,
                 val_prec_gb_orig, val_prec_gb_smote, val_prec_gb_rus],
    'Recall': [val_rec_lr_orig, val_rec_lr_smote, val_rec_lr_rus,
              val_rec_rf_orig, val_rec_rf_smote, val_rec_rf_rus,
              val_rec_gb_orig, val_rec_gb_smote, val_rec_gb_rus],
    'F1-Score': [val_f1_lr_orig, val_f1_lr_smote, val_f1_lr_rus,
                val_f1_rf_orig, val_f1_rf_smote, val_f1_rf_rus,
                val_f1_gb_orig, val_f1_gb_smote, val_f1_gb_rus],
    'AUC': [val_auc_lr_orig, val_auc_lr_smote, val_auc_lr_rus,
           val_auc_rf_orig, val_auc_rf_smote, val_auc_rf_rus,
           val_auc_gb_orig, val_auc_gb_smote, val_auc_gb_rus]
})

# Mostrar la tabla de resultados
val_results.style.background_gradient(cmap='Blues')

In [ ]:
# Visualizar comparación de F1-Score entre modelos (validación)
plt.figure(figsize=(16, 10))
ax = sns.barplot(x='Modelo', y='F1-Score', data=val_results, palette='viridis')
plt.title('Comparación de F1-Score entre Modelos (Validación)', fontsize=18)
plt.xlabel('Modelo', fontsize=14)
plt.ylabel('F1-Score', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.ylim(0, 1.1)
plt.grid(True, linestyle='--', alpha=0.7)

# Añadir etiquetas con los valores
for i, p in enumerate(ax.patches):
    ax.annotate(f'{p.get_height():.4f}', 
                (p.get_x() + p.get_width() / 2., p.get_height() + 0.01), 
                ha = 'center', va = 'bottom', fontsize=12, rotation=0)
    
plt.tight_layout()
plt.show()

## 10. Conclusiones

### Hallazgos Principales

1. **Alta Predictibilidad**: Los eventos de inundación pueden ser predichos con extraordinaria precisión utilizando las variables disponibles, particularmente con modelos de ensamble como Random Forest y Gradient Boosting.

2. **Variables Críticas**: El caudal (`cfs`) y la precipitación (`p01m`) son los indicadores más importantes para predecir eventos de inundación, representando aproximadamente el 99% de la importancia total de las características.

3. **Efectividad de Modelos Complejos**: Los modelos basados en árboles superaron significativamente a la Regresión Logística, especialmente en el manejo del desbalance de clases.

4. **Estrategias de Balanceo**: El uso de técnicas como SMOTE y RandomUnderSampler mostró resultados variables, pero en general SMOTE tendió a producir modelos con mejor rendimiento en términos de F1-Score.

5. **Robustez del Modelo**: La validación cruzada y la evaluación en conjuntos separados (train/validation/test) confirman la estabilidad y confiabilidad de nuestro modelo final.

### Recomendaciones

1. **Implementar un Sistema de Alerta Temprana**: Utilizar el modelo seleccionado para predecir eventos de inundación con anticipación.

2. **Incorporar Variables Adicionales**: Considerar la inclusión de otras variables meteorológicas y geográficas para mejorar aún más el modelo.

3. **Explorar Modelos Temporales Avanzados**: Evaluar el uso de modelos específicos para series temporales como LSTM o GRU para capturar mejor la dinámica temporal.

4. **Monitoreo Continuo**: Establecer un proceso de monitoreo y actualización regular del modelo con nuevos datos.